In [ ]:
!pip install keras tensorflow-serving-api

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential

batch_size = 128
num_classes = 10
epochs = 20

sess = tf.Session()

from keras import backend as K
K.set_session(sess)

In [ ]:
# this placeholder will contain our input digits, as flat vectors
img = tf.placeholder(tf.float32, shape=(None, 784))

In [ ]:
from keras.layers import Dense, InputLayer
from keras.models import Model, Sequential
from keras.optimizers import RMSprop

# Keras layers can be called on TensorFlow tensors:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=784))  # fully-connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='softmax'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
from keras.datasets import mnist
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#from tensorflow.examples.tutorials.mnist import input_data
#mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)



# Initialize all variables
#init_op = tf.global_variables_initializer()
#sess.run(init_op)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Run training loop
#with sess.as_default():
#    for i in range(100):
#        batch = mnist_data.train.next_batch(50)
#        train_step.run(feed_dict={img: batch[0],
#                                  labels: batch[1]})

In [ ]:
print(y_test[800])
print(x_test[800])

In [ ]:
from keras import backend as K
K.set_learning_phase(0)  # all new operations will be in test mode from now on

# serialize the model and get its weights, for quick re-building
config = model.get_config()
weights = model.get_weights()

# re-build a model where the learning phase is now hard-coded to 0
from keras.models import model_from_config
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

new_model = Sequential.from_config(config)
new_model.set_weights(weights)

In [ ]:
export_path = "gs://mkt-cloudml-jumpstart-johanna/model1"
#def to_savedmodel(model, export_path):
"""Convert the Keras HDF5 model into TensorFlow SavedModel."""

builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'input': model.inputs[0]},
                                    outputs={'income': model.outputs[0]})

with K.get_session() as sess:
   builder.add_meta_graph_and_variables(
        sess=sess,
        tags=[tag_constants.SERVING],
        signature_def_map={
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature}
   )
builder.save()

In [ ]:
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors

credentials = GoogleCredentials.get_application_default()

ml = discovery.build('ml','v1', credentials=credentials)
projectID = 'projects/{}'.format('mkt-cloudml-jumpstart')

In [ ]:
# Create ml_engine model
model_name = 'mnist_keras_test_johanna'
requestDict = {'name': model_name,
               'description': 'Keras mnist classifier'}
try:
    response = ml.projects().models().create(parent=projectID,
                                         body=requestDict).execute()
    ml_model = response
    print(response)
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

In [ ]:

# Create ml_engine model version
requestDict = { "name": "v1",
                "description": "Version 1",
                "deploymentUri": export_path
              }

model_path = projectID + "/models/" + model_name
try:
    response = ml.projects().models().versions().create(parent=model_path ,
                                         body=requestDict).execute()
    print(response)
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error creating the model version. Check the details:')
    print(err._get_reason())

In [ ]:
# Do online predition

requestDict = { "instances": [ x_test[800].tolist() ]
              }
#print(requestDict)
#model_path = projectID + "/models/" + model_name
try:
    response = ml.projects().predict(name=model_path ,
                                         body=requestDict).execute()
    print(response)
except errors.HttpError, err:
    # Something went wrong, print out some information.
    print('There was an error predicting. Check the details:')
    print(err._get_reason())

In [ ]:
results_list = response["predictions"][0]["income"]
index_prediction = results_list.index(max(results_list))
print(results_list)
print('Predicted digit: ' + str(index_prediction) + ' with a confidence of: ' + str(results_list[index]))
print('------------------')
print(y_test[800])

In [ ]:
from random import randint

for x in range(0, 100):
    i = randint(0, 9999)
    requestDict = { "instances": [ x_test[i].tolist() ]
                  }
                   
    try:
        response = ml.projects().predict(name=model_path ,
                                         body=requestDict).execute()
        results_list = response["predictions"][0]["income"]
        index = results_list.index(max(results_list))
        print('For index: ' + str(i) + ' predicted digit is: ' + str(index) + ' with a confidence of: ' + str(results_list[index]))
    except errors.HttpError, err:
        # Something went wrong, print out some information.
        print('There was an error predicting. Check the details:')
        print(err._get_reason())